In [9]:
import numpy as np
import multiprocessing as mp

from functools import partial
from matplotlib import pyplot as plt

In [37]:
"""
Gera uma reta aleatória
"""
def generate_line():
    
    # Dimensões do poblema
    d = 2
    
    # Gera 2 pontos aleatórios entre 0 e 1 e os converte para
    # o intervalo entre -1 e 1
    p1 = np.random.rand(d) * 2 - 1
    p2 = np.random.rand(d) * 2 - 1
    
    # Calcula a reta na forma `y=ax + b` entre os pontos
    a = (p2[1] - p1[1]) / (p2[0] - p1[0])
    b = p1[1] - a*p1[0]

    return np.array([b, a, -1])

"""
Gera os dados baseados em uma reta
"""
def generate_data(line, n):
    
    # Gera 'n' pontos (x, y) aleatoriamente entre 0 e 1
    # e os converte para o intervalo -1 e 1
    points = (np.random.rand(n, 2) * 2) - 1
    
    return points

"""
Calcula os valores de saída da função ideal
"""
def calc_y(line, data):
    
    y = line[0] + np.dot(data, line[1:])
    
    # Transforma os valores em -1 ou 1
    # Valores em cima da reta são considerados -1
    y = np.where(y > 0, 1, -1)
    
    return y

In [64]:
"""
Executa o perceptron para um conjunto de dados e de pesos iniciais
Caso os pesos não sejam especificados, eles serão o vetor nulo
"""
def perceptron(data, y, weights=None):
    if weights is None:
        # Cria um vetor de zeros do tamanho da quantidade de dimensões + viés
        weights = np.zeros(data.shape[1]+1 if len(data.shape) > 1 else 1)

    # Contador de iterações
    it = 0

    # Adiciona a coluna do viés ao X
    m_data = np.concatenate((np.ones((data.shape[0], 1)), data), axis=1)
    
    # Vetor que indica quais pontos estão classificados erroneamente
    # Inicialmente começa com 1(True) para todos os valores,
    # pois nenhum dado foi classificado
    miscl_mask = np.ones(m_data.shape[0], dtype=bool)
    
    # Filtra os dados para que não foram classificados
    miscl_data = m_data[miscl_mask]
    
    # Executar até que não haja mais pontos classificados erroneamente
    while miscl_data.shape[0] > 0:
        
        # Filtra os dados de saída não classificados
        miscl_y = y[miscl_mask]

        # Seleciona um ponto aleatório dentro o conjunto dos não classificados
        i = np.random.randint(miscl_data.shape[0])

        # Atualiza os pesos
        weights += miscl_y[i] * miscl_data[i]

        # Calcula as saídas com os novos pesos
        temp = np.sign(np.dot(m_data, weights))
        
        # Atualiza os pontos não classificados
        # Caso os valores calculados sejam diferentes dos corretos
        # a subtração vai dar um valor diferente de 0
        # que quando convertido para um tipo 'bool' valerá 'True'
        # ou seja, classificado erroneamente
        miscl_mask = np.array( temp - y, dtype=bool)
        
        # Atualiza os dados classificados erroneamente
        miscl_data = m_data[miscl_mask]
        
        # Atualiza a iteração do algoritmo
        it += 1
        
    # Retorna os pesos finais e o número de iterações
    return weights, it

In [39]:
def lin_regression(x, y):
    
    m_x = np.concatenate((np.ones((x.shape[0], 1)), x), axis=1)
    
    x_dagger = np.dot( np.linalg.inv( np.dot(m_x.T , m_x) ), m_x.T)

    w_lin = np.dot(x_dagger, y)
    
    return w_lin

In [66]:
def calc_error(line, w):
    
    # Gera mil pontos para serem avaliados
    ev_data = generate_data(line, 1000)
    
    # Saída ideal
    ev_f_y = calc_y(line, ev_data)
    
    # Saída gerada pela g(x)
    ev_g_y = calc_y(w, ev_data)
    
    # Conta todos os pontos em que as saídas não foram iguais
    misclassified = np.count_nonzero(ev_f_y - ev_g_y)

    # Calcula a porcentagem dos pontos classificados erroneamente
    return misclassified / ev_data.shape[0]


def experiment(N):
    line = generate_line()

    x = generate_data(line, N)
    y = calc_y(line, x)
    
    w_lin = lin_regression(x, y)
    
    y_g = calc_y(w_lin, x)

    e_in = np.count_nonzero( y - y_g ) / y.shape[0]
    
    e_out = calc_error(line, w_lin)
    
    w_perc, it = perceptron(x, y, w_lin)
    
    return [e_in, e_out, it]

In [69]:
"""
Executa um certo número de experimentos paralelamente
Caso o número de processos não seja espeficidado,
o multiprocessing utiliza o valor padrão,
que costuma ser o número de processadores
"""
def run_experiment(N, num_exp, processes=None):
    pool = mp.Pool(processes)
    
    # Executa os experimentos 'num_exp' vezes, passando como
    # parâmetro para cada um, o número de dados N a serem gerados
    results = np.array(pool.map(experiment, [N] * num_exp))
    
    # Calcula a média dos resuldados por coluna (iterações, erro)
    return np.mean(results, axis=0)

In [70]:
result_1, result_2 = run_experiment(100, 1000)[0,1]
result_3 = run_experiment(10, 1000)[2]

print('Question 1:', result_1)
print('Question 2:', result_2)
print('Question 2:', result_3)

Question 1: 0.03872
Question 2: 0.050184
Question 2: 6.959
